<a href="https://colab.research.google.com/github/hatopopvr/MyBeatSaberAnalytics/blob/main/MyBeatSaberAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Readme

### 概要
BeatSaberのScoreSaberの情報を取得し、Rank譜面Clearの進捗状況や、Play傾向を散布図で分析するツールです。

本ツールは [Satoさん](https://twitter.com/zitasato) の [MyBeatSaberScore](https://github.com/tkns3/MyBeatSaberScore/) に影響を受け、クリア進捗やスコア傾向をグラフ等で可視化するために作成しました。  本ツールには未クリア譜面の詳細検索や未クリア譜面のプレイリスト作成機能などは含まれません。

### 使用方法

本notebookをGoogle Colobで起動してください。
入力情報のフォームに必要事項を入力し、ランタイムでセルを実行してください。
セルを実行していただくと、Google Driveにマウントし、曲情報やPlay情報などをGoogle Driveの指定フォルダに保存します。
プレイ情報と曲情報の取得・解析には時間がかかります。

Rank譜面のClearの進捗状況や、SS/S/AなどのScoreのランクの状況を可視化できます。

<img src="https://pbs.twimg.com/media/FPBJ8_xacAU9rNA?format=jpg" />

条件を選択し散布図で分析できます。

<img src="https://pbs.twimg.com/media/FO8HJb1agAcVvn5?format=jpg" />


### 使用データ
本ツールは以下のデータを利用しています。
- ScoreSaberのPublic API - [doc](https://docs.scoresaber.com/)  
　Player情報 - https://scoresaber.com/api/player/{player_id}/full  
　Score情報 - https://scoresaber.com/api/player/{player_id}/scores?sort=recent&page={page}  
- 画像データ  
　Player画像 - https://cdn.scoresaber.com/avatars/{player_id}.jpg  
　Cover画像 - https://cdn.scoresaber.com/covers/{hash}.png  
- 曲情報  
  暫定的ですがandruzzzhkaさんがgithubで公開している[
BeatSaberScrappedData](https://github.com/andruzzzhka/BeatSaberScrappedData) を利用しています。

### 使用ライブラリ

本ツールは多数のOSSのLibraryに依存しています。
使用項目については Libraryの取得 を参照願います。

### 問い合わせ先

twitter:[@hatopop_vr](https://twitter.com/hatopop_vr)

### 使用ライブラリ

In [1]:
!pip install GitPython

In [2]:
#@title Libraryの取得
import os
import shutil
import pandas as pd
from pandas import json_normalize
import json
from datetime import datetime
from dateutil import tz
#import matplotlib
#import seaborn as sns
import plotly.express as px
from ipywidgets import interact, Select, BoundedIntText, IntSlider, ToggleButtons, Layout, HBox, VBox, AppLayout
import warnings
import time
from tqdm import tqdm
import requests
import math
import git
import gc
from google.colab import drive
warnings.filterwarnings('ignore')

# 入力情報

## 入力変数
- player_id: ScoreSaberのPlayerIDです。  
- google_drive_mount_path: GoogleDriveのマウント先です。変更不要。  
- google_drive_dir_path: GoogleDrive内の保存先です。必要に応じて変更してください。  
- song_data_zip_git_url: ScoreSaberの全曲情報のzip(json)のURLです。変更なければそのままで。  
- latest:Latest Historyで表示する対象の日数です。0なら現在から24時間以内。  
- saved_song_list_is_enable:解析済の曲情報を使用するか。☑で使用。※新規取得は数十分要。
- saved_player_score_is_enable:
解析済のプレイ情報を使用するか。☑で使用。※処理時間は総PlayCountに依存。  

In [3]:
#@title 変数概要を参照し、以下のデータを入力してください。 { display-mode: "form" }
player_id =  76561198412839195#@param {type:"number"}
google_drive_mount_path = "/content/drive" #@param {type:"string"}
google_drive_dir_path = "/MyDrive/MyBeatSaberAnalysis/data" #@param {type:"string"}
song_data_zip_git_url = "https://github.com/andruzzzhka/BeatSaberScrappedData.git" #@param {type:"string"}
latest =  1#@param {type:"integer"}
saved_song_list_is_enable = False #@param {type:"boolean"}
saved_player_score_is_enable = True #@param {type:"boolean"}

In [4]:
#@title Google Driveのマウント
drive.mount(google_drive_mount_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#@title その他設定値
data_path = r'{}{}'.format(google_drive_mount_path, google_drive_dir_path)
player_score_path = r'{}/scores_full_{}.csv'.format(data_path, player_id)
player_ranked_path = r'{}/scores_ranked_{}.csv'.format(data_path, player_id)
song_clone_path = r'/content/BeatSaberScrappedData'
song_zip_path = r'{}/combinedScrappedData.zip'.format(song_clone_path)
song_json_path = r'{}/combinedScrappedData.json'.format(data_path)
song_list_path = r'{}/song_list_full.csv'.format(data_path)
song_ranked_path = r'{}/song_ranked.csv'.format(data_path)
level_cleared_path = r'{}/level_cleared_{}.csv'.format(data_path, player_id)
pd.options.display.precision = 2
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
page_count=8 #ScoreSaberの1ページあたりのページ数
image_size=50 #カバー,プロファイル画像のサイズ単位px
os.makedirs(data_path, exist_ok=True)

In [31]:
#@title 列情報の設定
cols_info =[
    'Pic'
    ,'name'
    ,'country'
    ,'pp'
    ,'rank'
    ,'role'
    ,'TotalScore'
    ,'RankedScore'
    ,'AveRankedAcc'
    ,'TotalPlay'
    ,'RankedPlay'
    ,'ReplayWatched'
]

cols =[
    'Cover'
    ,'Song'
    ,'Level'
    ,'Star'
    ,'Acc'
    ,'AccRank'
    ,'Rank'
    ,'PP'
    ,'Miss'
    ,'Bad'
    ,'Combo'
    ,'Score'
    ,'Mode'
    ,'Difficulty'
    ,'Play'
    ,'PlayD'
    ,'Days'
 ]

cols_xy = [
    'Star'
    ,'Acc'
    ,'Rank'
    ,'PP'
    ,'Miss'
    ,'Bad'
    ,'Combo'
    ,'Score'
    ,'Play'
    ,'PlayD'
    ,'Days'
    ,'Months'
]

cols_cate = [
    'LevelS'
    ,'DaysS'
    ,'MonthsS'
    ,'LatestS'
    ,'Acc'
    ,'AccRank'
]

cols_song =[
    'Cover'
    ,'Song'
    ,'Level'
    ,'Star'
    ,'maxPP'
    ,'maxScore'
    ,'Difficulty'
    ,'Play'
    ,'PlayD'
    ,'CreatedDateJa'
    ,'RankDateJa'
]

cols_clear_x = [
    'Level'
]

cols_clear_y = [
    'AlreadyCleared'
    ,'RecentCleared'
    ,'NotCleared'
]

cols_rate_y = [
    'AlreadyClearedRate'
    ,'RecentClearedRate'
    ,'NotClearedRate'
]

cols_acc_rank = [
    'SSS'
    ,'SS'
    ,'S'
    ,'A'
    ,'B'
    ,'C'
    ,'D'
    ,'E'
    ,'NotCleared'
]

cols_accs = ['SSS','SS','S','A','B']

cols_pair = [
    'Level'
    ,'Star'
    ,'Acc'
    ,'Rank'
    ,'PP'
    ,'Miss'
    ,'Bad'
    ,'Combo'
    ,'Score'
    ,'Play'
    ,'PlayD'
    ,'Days'
    ,'Months'
    ,'Modifiers' 
    ,'LevelS'
    ,'DaysS'
    ,'MonthsS'
]

cols_song =[
    'Cover'
    ,'Song'
    ,'Level'
    ,'LevelS'
    ,'Star'
    # ,'maxPP'
    # ,'maxScore'
    ,'Difficulty'
    # ,'Play'
    # ,'PlayD'
    ,'CreatedDateJa'
    ,'RankDateJa'
    ,'Bombs'
    ,'Notes'
    ,'Obstacles'
    ,'Njs'
    ,'NjsOffset'
    ,'Bpm'
    ,'Upvotes'
    #,'Downvotes'
    ,'Duration'
    ,'Ranked'
]


# データの取得

## Get Player Info

In [7]:
#@title Player情報の取得
url = r"https://scoresaber.com/api/player/{}/full".format(player_id)
response = requests.get(url)
res_data = response.json()
df_info = json_normalize(res_data)

df_info['Pic'] = '<img src="'+df_info['profilePicture']+'"/>'
df_info['TotalScore'] = df_info['scoreStats.totalScore']
df_info['RankedScore'] = df_info['scoreStats.totalRankedScore']
df_info['AveRankedAcc'] = df_info['scoreStats.averageRankedAccuracy']
df_info['TotalPlay'] = df_info['scoreStats.totalPlayCount']
df_info['RankedPlay'] = df_info['scoreStats.rankedPlayCount']
df_info['ReplayWatched'] = df_info['scoreStats.replaysWatched']

cols_info =[
    'Pic'
    ,'name'
    ,'country'
    ,'pp'
    ,'rank'
    ,'role'
    ,'TotalScore'
    ,'RankedScore'
    ,'AveRankedAcc'
    ,'TotalPlay'
    ,'RankedPlay'
    ,'ReplayWatched'
]

PlayCount = df_info['TotalPlay'][0]
RangeCount = math.ceil(PlayCount / page_count) + 1
print("Play数:{}, Page数:{}".format(PlayCount, RangeCount))

Play数:786, Page数:100


In [8]:
df_info[cols_info].style.highlight_null(null_color='pink').set_precision(2)

,Pic,name,country,pp,rank,role,TotalScore,RankedScore,AveRankedAcc,TotalPlay,RankedPlay,ReplayWatched
0,,hatopop,JP,5647.96,5242,None,627823284,606309296,80.27,786,749,0


## Get Player Scores

In [9]:
#@title Score情報の取得
if saved_player_score_is_enable and os.path.exists(player_score_path):    
    df = pd.read_csv(player_score_path)

else:
    url = r"https://scoresaber.com/api/player/{}/scores?sort=recent".format(player_id)
    response = requests.get(url)
    res_data = response.json()
    df = json_normalize(res_data['playerScores'])

    for i in tqdm(range(2, RangeCount)):
        url = r"https://scoresaber.com/api/player/{}/scores?sort=recent&page={}".format(player_id, i)
        try:
            response = requests.get(url)
            res_data = response.json()
            df2 = json_normalize(res_data['playerScores'])
            df=df.append(df2, ignore_index=True)
        except:
            break

    df['Song'] = df['leaderboard.songName'] + " " + df['leaderboard.songSubName'] + " / " + df['leaderboard.songAuthorName'] + " [" + df['leaderboard.levelAuthorName'] + "]"
    df['Acc'] = df['score.modifiedScore'] / df['leaderboard.maxScore'] * 100
    df['Mode'] = df['leaderboard.difficulty.gameMode']
    df_ = df['leaderboard.difficulty.difficultyRaw'].str.split('_', expand=True)
    df_.columns = ['_','Difficulty', 'Mode']
    df['Difficulty'] = df_['Difficulty']
    df['Star'] = df['leaderboard.stars']
    df['Level'] = df['Star'].astype('int')
    df['LevelS'] = df['Level'].astype('str')
    df['Score'] = df['score.modifiedScore']
    df['Bad'] = df['score.badCuts']
    df['Miss'] = df['score.missedNotes']
    df['Combo'] = df['score.maxCombo']
    df['PP'] = df['score.pp']
    df['PPWeight'] = df['score.pp'] * df['score.weight']
    df['Rank'] = df['score.rank']
    df['Modifiers'] = df['score.modifiers']
    df['Play'] = df['leaderboard.plays']
    df['PlayD'] = df['leaderboard.dailyPlays']
    df['DateUtc'] = pd.to_datetime(df['score.timeSet'])
    df_idx = df.set_index('DateUtc')
    df['DateJa'] = df_idx.index.tz_convert('Asia/Tokyo')
    df = df.set_index('DateJa')
    df['Days'] = (pd.Timestamp(datetime.now(), tz='Asia/Tokyo') - df.index).days
    df['Months'] = (df['Days'] / 30).astype('int')
    df['DaysS'] = df['Days'].astype('str')
    df['MonthsS'] = df['Months'].astype('str')
    def func_open(x):
        if  x <= latest:
            return 1
        else:
            return 0

    df['Latest'] = df['Days'].apply(func_open)
    
    def func_score(x):
        if  x >= 100:
            return "SSS"
        elif x >= 90:
            return "SS"
        elif x >= 80:
            return "S"
        elif x >= 65:
            return "A"
        elif x >= 50:
            return "B"
        elif x >= 35:
            return "C"
        elif x >= 20:
            return "D"
        elif x >= 0:
            return "E"
        else:
            return "Error"

    df['AccRank'] = df['Acc'].apply(func_score)
  
    df['Cover'] = '<img src="'+df['leaderboard.coverImage']+'" style="width:{}px;"/>'.format(image_size)

    #SongListの保存
    df[(df['leaderboard.ranked'] == True)][cols].sort_index(ascending=False).to_csv(player_ranked_path)
    df.sort_index(ascending=False).to_csv(player_score_path.format(player_id))

print('ScoreCount:{}, LatestPlayCount:{}'.format(df['Play'].count(), df[(df['Latest'] == 1)]['Play'].count()))

ScoreCount:786, LatestPlayCount:117


## Get Song List

In [10]:
#@title Song Listを取得し展開(最新取得は30分程度かかる可能性あり)
if saved_song_list_is_enable and os.path.exists(song_list_path):
    df_song = pd.read_csv(song_list_path)

else:
    # GithubからZipのダウンロード
    if os.path.exists(song_clone_path):
        shutil.rmtree(song_clone_path)

    git.Git().clone(song_data_zip_git_url)

    # Zipの解凍
    shutil.unpack_archive(song_zip_path, data_path)

    json_open = open(song_json_path,'r', encoding="utf-8")
    json_load = json.load(json_open)
    df_song_ = json_normalize(json_load)

    cnt=0

    df_s_ = df_song_.loc[0]

    _df = json_normalize(df_s_['Diffs'])
    _df['Key']=df_s_['Key']
    _df['Hash']=df_s_['Hash']
    _df['SongName']=df_s_['SongName']
    _df['SongSubName']=df_s_['SongSubName']
    _df['SongAuthorName']=df_s_['SongAuthorName']
    _df['LevelAuthorName']=df_s_['LevelAuthorName']
    _df['Chars']=df_s_['Chars'][0]
    _df['Uploaded']=df_s_['Uploaded']
    _df['Uploader']=df_s_['Uploader']
    _df['Bpm']=df_s_['Bpm']
    _df['Upvotes']=df_s_['Upvotes']
    _df['Downvotes']=df_s_['Downvotes']
    _df['Duration']=df_s_['Duration']

    df_song = _df

    for idx in tqdm(range(len(df_song_))):

        if cnt == 0:
            cnt+=1
            pass
        cnt+=1
        _df = json_normalize(df_song_.loc[idx]['Diffs'])
        _df['Key']=df_song_.loc[idx]['Key']
        _df['Hash']=df_song_.loc[idx]['Hash']
        _df['SongName']=df_song_.loc[idx]['SongName']
        _df['SongSubName']=df_song_.loc[idx]['SongSubName']
        _df['SongAuthorName']=df_song_.loc[idx]['SongAuthorName']
        _df['LevelAuthorName']=df_song_.loc[idx]['LevelAuthorName']
        _df['Chars']=df_song_.loc[idx]['Chars'][0]
        _df['Uploaded']=df_song_.loc[idx]['Uploaded']
        _df['Uploader']=df_song_.loc[idx]['Uploader']
        _df['Bpm']=df_song_.loc[idx]['Bpm']
        _df['Upvotes']=df_song_.loc[idx]['Upvotes']
        _df['Downvotes']=df_song_.loc[idx]['Downvotes']
        _df['Duration']=df_song_.loc[idx]['Duration']
        df_song = df_song.append(_df, ignore_index=True)

    df_song['Song'] = df_song['SongName'] + " " + df_song['SongSubName'] + " / " + df_song['SongAuthorName'] + " [" + df_song['LevelAuthorName'] + "]"
    df_song['Difficulty'] = df_song['Diff']
    df_song['Star'] = df_song['Stars']
    df_song['Level'] = df_song['Star'].astype('int')
    df_song['LevelS'] = df_song['Level'].astype('str')
    df_song['Cover'] = '<img src="https://cdn.scoresaber.com/covers/' + df_song['Hash'] + '.png" style="width:{}px;"/>'.format(image_size)
    df_song['RankDateUtc'] = pd.to_datetime(df_song['RankedUpdateTime'])
    df_song_idx = df_song.set_index('RankDateUtc')
    df_song['RankDateJa'] = df_song_idx.index.tz_convert('Asia/Tokyo')
    df_song = df_song.set_index('RankDateJa')
    df_song = df_song.reset_index()
    df_song['CreatedUtc'] = pd.to_datetime(df_song['Uploaded'])
    df_song_idx = df_song.set_index('CreatedUtc')
    df_song['CreatedDateJa'] = df_song_idx.index.tz_convert('Asia/Tokyo')
    df_song = df_song.reset_index()
    
    df_song[df_song['Ranked']==1][cols_song].to_csv(song_ranked_path, index=False)
    df_song.to_csv(song_list_path, index=False)

print('Song:{}, RankedSong:{}'.format(df_song['Song'].count(), df_song[df_song['Ranked']==1]['Song'].count()))

100%|██████████| 59538/59538 [57:50<00:00, 17.16it/s]


Song:93687, RankedSong:3083


## Ranked - Calc Level Cleared Progress

In [11]:
#@title Level別 Clear 情報の結合
df_merge = df_song[df_song['Ranked']==1][['Level','Song']].groupby(['Level'],as_index=False).count()
df_cleared = df[(df['Modifiers'] != 'NF')
                & (df['leaderboard.ranked'] == True)
                ][['Level','Song']].groupby(['Level'],as_index=False).count().rename(columns={'Song': 'Cleared'})
df_cleared_latest = df[(df['Modifiers'] != 'NF') 
                       & (df['leaderboard.ranked'] == True)
                       & (df['Latest'] == 1)
                ][['Level','Song']].groupby(['Level'],as_index=False).count().rename(columns={'Song': 'RecentCleared'})
df_merge = pd.merge(df_merge, df_cleared_latest, on='Level', how='outer')
df_merge = pd.merge(df_merge, df_cleared, on='Level', how='outer')

df_merge['NotCleared'] = df_merge['Song'].fillna(0) - df_merge['Cleared'].fillna(0)
df_merge['AlreadyCleared'] = df_merge['Cleared'].fillna(0) - df_merge['RecentCleared'].fillna(0)
df_merge = df_merge.fillna(0)
df_merge['NotClearedRate'] = df_merge['NotCleared'] / df_merge['Song'] * 100
df_merge['AlreadyClearedRate'] = df_merge['AlreadyCleared'] / df_merge['Song'] * 100
df_merge['RecentClearedRate'] = df_merge['RecentCleared'] / df_merge['Song'] * 100



df_acc =df[(df['Modifiers'] != 'NF')
                & (df['leaderboard.ranked'] == True)
                ].pivot_table(index="Level", columns= "AccRank", values="Song", aggfunc='count').fillna(0).reset_index()

for cols_acc in cols_accs:
    if cols_acc in df_acc.columns :
        pass
    else:
        print('Rank{}データ無し'.format(cols_acc))
        df_acc[cols_acc] = 0

df_merge = pd.merge(df_merge, df_acc, on='Level', how='outer').fillna(0)

df_merge['Other'] = df_merge['Song'] - df_merge['SS'] - df_merge['S'] - df_merge['A'] - df_merge['B']
df_merge['SS-Rate'] = df_merge['SS'] / df_merge['Song'] * 100
df_merge['S-Rate'] = df_merge['S'] / df_merge['Song'] * 100
df_merge['A-Rate'] = df_merge['A'] / df_merge['Song'] * 100
df_merge['B-Rate'] = df_merge['B'] / df_merge['Song'] * 100
df_merge['Other-Rate'] = df_merge['Other'] / df_merge['Song'] * 100

df_merge.to_csv(level_cleared_path, index=False)

RankSSSデータ無し


# 分析 可視化

### Latest History

In [12]:
#@title history filter
filtered_level_min = 0 #@param {type:"slider", min:0, max:15, step:1}
filtered_level_max = 12 #@param {type:"slider", min:0, max:15, step:1}
filtered_pp_min = 0 #@param {type:"slider", min:0, max:600, step:10}
filtered_pp_max = 600 #@param {type:"slider", min:0, max:600, step:10}

df[(1==1)
    & (df['Latest'] == 1)
    & (df['Modifiers'] != 'NF')
    & (df['Star'] >= filtered_level_min)
    & (df['Star'] < filtered_level_max)
    & (df['PP'] >= filtered_pp_min)
    & (df['PP'] < filtered_pp_max)
][cols].sort_index(ascending=False).style.highlight_null(null_color='red').set_precision(2)

,Cover,Song,Level,Star,Acc,AccRank,Rank,PP,Miss,Bad,Combo,Score,Mode,Difficulty,Play,PlayD,Days
116,,Chapter 99: Afterword / t+pazolite [Emilia],1,1.45,95.48,SS,374,63.87,0,0,107,87071,SoloStandard,Easy,727,3,1
115,,Chapter 99: Afterword / t+pazolite [Emilia],0,0.98,95.40,SS,275,42.74,0,0,149,123857,SoloStandard,Normal,697,2,1
114,,In my room / FELT [Kival Evan],0,0.94,96.99,SS,421,51.81,0,0,34,23311,SoloStandard,Easy,769,2,1
113,,In my room / FELT [Kival Evan],1,1.11,96.50,SS,273,56.16,0,0,55,41836,SoloStandard,Normal,607,3,1
112,,Ending / Thaehan [FatBeanzoop],1,1.55,95.39,SS,633,67.42,0,0,73,57151,SoloStandard,Easy,1206,6,1
111,,Ending / Thaehan [FatBeanzoop],1,1.20,95.54,SS,388,53.23,0,0,97,78336,SoloStandard,Normal,1033,4,1
110,,Ending / Thaehan [FatBeanzoop],2,2.07,93.34,SS,538,78.95,0,0,146,118606,SoloStandard,Hard,1512,7,1
109,,Ending / Thaehan [FatBeanzoop],4,4.46,82.33,S,1588,95.20,3,2,114,173550,SoloStandard,Expert,2248,10,1
108,,Naughty little demon (ED version) / Minari [Hoppaw],1,1.13,95.65,SS,422,50.67,0,0,21,11550,SoloStandard,Easy,592,6,1
107,,Naughty little demon (ED version) / Minari [Hoppaw],1,1.26,95.10,SS,256,53.55,0,0,43,30732,SoloStandard,Normal,429,4,1


## Ranked - Level Cleared Progress

In [13]:
#@title Ranked - Level Cleared Progress
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Cleared Count", "Cleared Rate", "Acc Ranked Count", "Acc Ranked Rate"))

fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['AlreadyCleared'], name='AlreadyCleared', marker_color='#636EFA'),row=1,col=1)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['RecentCleared'], name='RecentCleared', marker_color='#ff4500'),row=1,col=1)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['NotCleared'], name='NotCleared', marker_color='#dcdcdc'),row=1,col=1)

fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['AlreadyClearedRate'], name='AlreadyClearedRate', marker_color='#636EFA'),row=1,col=2)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['RecentClearedRate'], name='RecentClearedRate', marker_color='#ff4500'),row=1,col=2)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['NotClearedRate'], name='NotClearedRate',marker_color='#dcdcdc'),row=1,col=2)

fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['SS'], name='SS', marker_color='#ff4500'),row=2,col=1)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['S'], name='S', marker_color='#ffaaff'),row=2,col=1)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['A'], name='A', marker_color='#ffcc66'),row=2,col=1)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['B'], name='B', marker_color='#fffd88'),row=2,col=1)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['Other'], name='Other',marker_color='#dcdcdc'),row=2,col=1)

fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['SS-Rate'], name='SS-Rate', marker_color='#ff4500'),row=2,col=2)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['S-Rate'], name='S-Rate', marker_color='#ffaaff'),row=2,col=2)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['A-Rate'], name='A-Rate', marker_color='#ffcc66'),row=2,col=2)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['B-Rate'], name='B-Rate', marker_color='#fffd88'),row=2,col=2)
fig.add_trace(go.Bar(x=df_merge['Level'], y=df_merge['Other-Rate'], name='Other-Rate',marker_color='#dcdcdc'),row=2,col=2)

fig.update_layout(barmode='relative', title_text='Ranked - Level Cleared Progress - {} - {}'.format(datetime.now().strftime('%Y.%m.%d'), df_info['name'][0]),width=1000, height=800)
fig.show()

## Ranked - PairPlot Analysis

In [36]:
#@title Ranked - PairPlot Analysis
def show_plot(col_x, col_y, size, color, pp_min, pp_max, star_min, star_max, latest):

    def func_open(x):
        if  x <= latest:
            return 'latest {}day'.format(latest)
        else:
            return 'old'.format(latest)

    df['LatestS'] = df['Days'].apply(func_open).astype('str')
      
    fig2 = px.scatter(df[(df['leaderboard.ranked']==1)
                       & (df['PP'] >= pp_min)
                       & (df['PP'] <= pp_max)
                       & (df['Star'] >= star_min)
                       & (df['Star'] <= star_max)
                                                 #& (df['Months'] <= 1)
                        ], x=col_x, y=col_y, color=color,
                     size=size, hover_data=['PP','Acc','AccRank','Score','Miss','Bad','Song'],
                     width=800, height=800)

    fig2.update_layout(title='Ranked - PairPlot Analysis - {} - {}'.format(datetime.now().strftime('%Y.%m.%d'), df_info['name'][0]), title_x=0.5,
                  xaxis_title = "x:{} (markersize:{}, markercolor:{})".format(col_x, size, color), yaxis_title="y:{}".format(col_y)
                  )
                  #font=dict(family="Times",size=18,color="black"))

    fig2.show()

w1 = ToggleButtons(description='X軸:', options=cols_xy)
w2 = ToggleButtons(description='Y軸:', options=cols_xy)
w3 = ToggleButtons(description='Size:', options=cols_xy)
w4 = ToggleButtons(description='Color:', options=cols_cate)

slider_star_min = IntSlider(
    value=0,
    min=0,
    max=15,
    step=1,
    description="Star min:"
)

slider_star_max = IntSlider(
    value=15,
    min=0,
    max=15,
    step=1,
    description="Star max:"
)

slider_pp_min = IntSlider(
    value=50,
    min=0,
    max=600,
    step=10,
    description="PP min:"
)

slider_pp_max = IntSlider(
    value=600,
    min=0,
    max=600,
    step=10,
    description="PP max:"
)

slider_latest = IntSlider(
    value=0,
    min=0,
    max=7,
    step=1,
    description="LatestFilter:"
)

interact(show_plot, col_x=w1, col_y=w2, size=w3, color=w4, pp_min=slider_pp_min, pp_max=slider_pp_max, star_min=slider_star_min, star_max=slider_star_max, latest=slider_latest)

interactive(children=(ToggleButtons(description='X軸:', options=('Star', 'Acc', 'Rank', 'PP', 'Miss', 'Bad', 'C…

<function __main__.show_plot>